In [128]:
library('dplyr')
library("zoo")
directory = '/media/nicolas/Elements/data/'
time_past = 1970
time_pre = 2010

In [129]:
openfile <- function(path_to_data,time){
    
    if (!file.exists(path_to_data)){
            stop("Invalid ID")
    }
    
    Data <- read.table(path_to_data,skip = 20,header=TRUE,sep=',')
    Data <- filter(Data,Data[,5]==0)               
    Data <- Data[,-c(1,2,5)]
    Data$DATE <- strptime(Data$DATE, format = "%Y%m%d")
 
    Data <- filter ( Data, Data$DATE$year > time - 1905 & Data$DATE$year <= time - 1900)
    if (!length(Data$DATE$year)==1826){
        stop('Unvailable data for this period')
    }
    return(Data)
}

In [130]:
quarterino <- function(STAID,time){
    
    prec_byquarter <- NULL
    temp_byquarter <- NULL
    
    path_to_temp = paste(directory,'TG/TG_STAID',STAID,'.txt',sep='')
    path_to_prec = paste(directory,'RR/RR_STAID',STAID,'.txt',sep='')
    
    data_temp <- openfile(path_to_temp,time)
    data_prec <- openfile(path_to_prec,time)
    
    #On extrait de la colonne de date un vecteur ne contenant que les années
    vec_year<-as.numeric(format(data_temp$DATE,"%Y"))
   #On extrait de la colonne de date un vecteur ne contenant que les mois
    vec_month<-as.numeric(format(data_temp$DATE, "%m"))
    
    #Calcule de la température moyenne de tout les mois de toutes les année : 
    data_temp_monthly_mean<-aggregate(data_temp$TG,by = list(vec_month),mean)
    
    #Calcule de la pluie cumulée de tout les mois de toutes les année :
    data_prec_annual_monthly_sum<-aggregate(data_prec$RR,by = list(vec_year,vec_month),sum)
    #Moyenne sur les 5 ans <- pas d'effet 'année exeptionnel' 
    data_prec_monthly_sum <- aggregate(data_prec_annual_monthly_sum$x,
                                   data_prec_annual_monthly_sum[2],mean)
    for ( i in c(1,4,7,10)){
        prec_byquarter <- slice(data_prec_monthly_sum,i:(i+2)) %>% select(x) %>% 
                                                                    sum %>% append(prec_byquarter)
        q <- slice(data_temp_monthly_mean,i:(i+2))
        q <-mean(q$x) 
        temp_byquarter <-append(temp_byquarter,q)
    }
    df <- data.frame(temp_byquarter,prec_byquarter)
    
    wettest <- filter (df,df$prec_byquarter==max(prec_byquarter))
    dryest <- filter (df,df$prec_byquarter==min(prec_byquarter))
    warmest <- filter (df,df$temp_byquarter==max(temp_byquarter))
    coldest <- filter (df,df$temp_byquarter==min(temp_byquarter))
    
    BIO8<- wettest[,1]
    BIO9<- dryest[,1]
    BIO10<- warmest[,1]
    BIO11<- coldest[,1]
    BIO16<- wettest[,2]
    BIO17<- dryest[,2]
    BIO18<- warmest[,2]
    BIO19<- coldest[,2]
    
    return(list(BIO8,BIO9,BIO10,BIO11,BIO16,BIO17,BIO18,BIO19))
    
}

In [133]:
r<-quarterino('000013',time_pre)
r
j<-quarterino('000013',time_past)
j

[[1]]
[1] 148.0143

[[2]]
[1] 52.2605

[[3]]
[1] 179.1585

[[4]]
[1] 21.77367

[[5]]
[1] 3421

[[6]]
[1] 1119.2

[[7]]
[1] 2480.4

[[8]]
[1] 1503.8

[[1]]
[1] 134.0262

[[2]]
[1] 42.65835

[[3]]
[1] 170.6073

[[4]]
[1] 13.6972

[[5]]
[1] 3858.2

[[6]]
[1] 1572

[[7]]
[1] 2640.2

[[8]]
[1] 1583